In [1]:
from sklearn.metrics import mutual_info_score
#from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, pearsonr
import seaborn as sns
import pandas as pd

from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, pearsonr
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import mutual_info_score
from scipy.stats import contingency
import pandas as pd
from scipy.stats.distributions import chi2

# Mathematical Underpinnings - Lab 4

Tests to verify hipoteses of independence (from Lab 4):

In [2]:
def independence_test(X, Y, stats):
    assert stats in ["chi2", "mi"]
    data = np.stack((X,Y),axis=1)
    discx = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    discx.fit(data)
    new_X = discx.transform(data)
    if stats == "chi2":
        chisq = chi2_contingency(pd.crosstab(new_X[:,0],new_X[:,1]))
        return chisq[0], chisq[1]
    else:
        MI = mutual_info_score(new_X[:,0], new_X[:,1])
        return MI, chi2.sf(2*X.shape[0]*MI,81)
    
def permutation_test(X, Y, B):
    benchmark = np.zeros(B)
    data = np.stack((X,Y),axis=1)
    discx = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    discx.fit(data)
    new_X = discx.transform(data)
    MI = mutual_info_score(new_X[:,0], new_X[:,1])
    for b in range(B):
        data[:,0] = np.random.permutation(X)
        discx = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
        discx.fit(data)
        new_X = discx.transform(data)
        benchmark[b] = mutual_info_score(new_X[:,0], new_X[:,1])
    return (1 + sum(benchmark >= MI)) / (B + 1)

## Task 1

In [3]:
def conditional_independence_test(X, Y, Z):
    data = np.stack((X,Y),axis=1)
    discx = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    discx.fit(data)
    new_X = discx.transform(data)
    n = X.shape[0]
    MI = 0
    for z in np.unique(Z):
        MI += np.count_nonzero(Z == z)/n * mutual_info_score(new_X[Z == z,0], new_X[Z == z,1])
    return MI, chi2.sf(2*n*MI,81*np.shape(np.unique(Z))[0])
    

In [4]:
# Conditionally independent
n = 1000
ro = 0.
data = np.random.multivariate_normal(np.zeros(2), np.array([[1, ro], [ro, 1]]), n)
conditional_independence_test(data[:,0], data[:,1], np.random.randint(0,2,n))

(0.08301572229910961, 0.3977753150081232)

In [5]:
# Conditionally dependent
n = 1000
ro = 0.15
X = np.random.randn(n)
Z = np.random.randint(0,2,n)
Y = Z * X + 1/10*np.random.randn(n) # noise added
conditional_independence_test(X,Y,Z)

(0.7067807919222464, 1.545732845611077e-198)

In [6]:
def conditional_permutation_test(X, Y, Z, B):
    benchmark = np.zeros(B)
    data = np.stack((X,Y),axis=1)
    discx = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    discx.fit(data)
    new_X = discx.transform(data)
    MI = 0
    for z in np.unique(Z):
        MI += np.count_nonzero(Z == z)/n * mutual_info_score(new_X[Z == z,0], new_X[Z == z,1])
    for b in range(B):
        for z in np.unique(Z):
            benchmark[b] += np.count_nonzero(Z == z)/n * mutual_info_score(np.random.permutation(new_X[Z == z,0]), new_X[Z == z,1])
       # benchmark[b] = mutual_info_score(new_X[:,0], new_X[:,1])
    return (1 + sum(benchmark >= MI)) / (B + 1)

In [7]:
# Conditionally independent
n = 1000
ro = 0.15
data = np.random.multivariate_normal(np.zeros(2), np.array([[1, ro], [ro, 1]]), n)
conditional_independence_test(data[:,0], data[:,1], np.random.randint(0,2,n))

(0.08394018283305553, 0.3595158487461059)

In [8]:
# Conditionally dependent
n = 1000
X = np.random.randn(n)
Z = np.random.randint(0,2,n)
Y = Z * X + 1/10*np.random.randn(n) # noise added
conditional_permutation_test(X,Y,Z, 100)

0.009900990099009901

## Task 2

In [9]:
def sample_from_model1(n = 1000):
    Z = 2*(np.random.randn(n) > 0) - 1
    X = 2*((Z/2 + np.random.randn(n)) > 0) - 1
    Y = 2*((Z/2 + np.random.randn(n)) > 0) - 1
    return X,Y,Z
    
def sample_from_model2(n =1000):
    X = 2*(np.random.randn(n) > 0) - 1
    Z = 2*(X/2 + np.random.randn(n) > 0) - 1
    Y = 2*(Z/2 + np.random.randn(n) > 0) - 1
    return X,Y,Z

def sample_from_model3(n = 1000):
    X = 2*(np.random.randn(n) > 0) - 1
    Y = 2*(np.random.randn(n) > 0) - 1
    Z = 2*(((X + Y)/2 + np.random.randn(n)) > 0) - 1
    return X,Y,Z

### a)

# Model 1: dependent, conditionally independent
# Model 2: dependent, conditionally independent
# Model 3: independent, conditionally dependent

### b)

In [10]:
X,Y,Z = sample_from_model1()
print("MI: "+ str(independence_test(X,Y, stats='mi')[0]))
print("CMI: "+ str(conditional_independence_test(X,Y, Z)[0]))

MI: 0.015172724527087844
CMI: 0.002043055551410947


In [11]:
X,Y,Z = sample_from_model2()
print("MI: "+ str(independence_test(X,Y, stats='mi')[0]))
print("CMI: "+ str(conditional_independence_test(X,Y, Z)[0]))

MI: 0.012861486888514517
CMI: 0.0009753020117778604


In [12]:
X,Y,Z = sample_from_model3()
print("MI: "+ str(independence_test(X,Y, stats='mi')[0]))
print("CMI: "+ str(conditional_independence_test(X,Y, Z)[0]))

MI: 0.00024150412125745202
CMI: 0.007514799009864512


### c)

# Tests


In [13]:
X,Y,Z = sample_from_model1()
print("Permutation test p-value: "+ str(permutation_test(X,Y,100)))
print("Conditional permutation test p-value: "+ str(conditional_permutation_test(X,Y, Z, 100)))

print("Chi2 test p-value: "+ str(independence_test(X,Y,stats="chi2")[1]))
print("Conditional chi2 test p-value: "+ str(conditional_independence_test(X,Y, Z)[1]))

Permutation test p-value: 0.009900990099009901
Conditional permutation test p-value: 0.0297029702970297
Chi2 test p-value: 1.5431800009248159e-12
Conditional chi2 test p-value: 1.0


In [14]:
# Model 1: dependent, conditionally independent - correct 

In [15]:
X,Y,Z = sample_from_model2()
print("Permutation test p-value: "+ str(permutation_test(X,Y,100)))
print("Conditional permutation test p-value: "+ str(conditional_permutation_test(X,Y, Z, 100)))

print("Chi2 test p-value: "+ str(independence_test(X,Y,stats="chi2")[1]))
print("Conditional chi2 test p-value: "+ str(conditional_independence_test(X,Y, Z)[1]))

Permutation test p-value: 0.009900990099009901
Conditional permutation test p-value: 0.31683168316831684
Chi2 test p-value: 0.00011378439499226755
Conditional chi2 test p-value: 1.0


In [16]:
# Model 2: dependent, conditionally independent - correct 

In [17]:
X,Y,Z = sample_from_model3(100000)
print("Permutation test p-value: "+ str(permutation_test(X,Y,100)))
print("Conditional permutation test p-value: "+ str(conditional_permutation_test(X,Y, Z, 100)))

print("Chi2 test p-value: "+ str(independence_test(X,Y,stats="chi2")[1]))
print("Conditional chi2 test p-value: "+ str(conditional_independence_test(X,Y, Z)[1]))

Permutation test p-value: 0.4158415841584158
Conditional permutation test p-value: 0.009900990099009901
Chi2 test p-value: 0.368633718953402
Conditional chi2 test p-value: 3.2017351184018677e-290


In [18]:
# Model 3: independent, conditionally dependent - correct, but conditional chi2 test works for large n only (slow convergence)

## d)

In [19]:
def sample_from_model4(n = 1000):
    X_cont = np.random.randn(n)
    Y_cont = np.random.randn(n)
    X = 2*(X_cont > 0) - 1
    Y = 2*(Y_cont > 0) - 1
    Z = 2*(((X + Y)/2 + np.random.randn(n)) > 0) - 1
    return X,Y,Z

In [20]:
X,Y,Z = sample_from_model4(100000)
print("Permutation test p-value: "+ str(permutation_test(X,Y,100)))
print("Conditional permutation test p-value: "+ str(conditional_permutation_test(X,Y, Z, 100)))

print("Chi2 test p-value: "+ str(independence_test(X,Y,stats="chi2")[1]))
print("Conditional chi2 test p-value: "+ str(conditional_independence_test(X,Y, Z)[1]))

Permutation test p-value: 0.9702970297029703
Conditional permutation test p-value: 0.009900990099009901
Chi2 test p-value: 0.928006500660194
Conditional chi2 test p-value: 4.713256218512558e-282


### Not much have changed